# Análisis

In [26]:
import pandas as pd 
import numpy as np 
import plotly.express as px
from sqlalchemy import create_engine
from decouple import Config,RepositoryEnv
import openpyxl
import warnings
# Desactivar advertencias de FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)

In [27]:
config = Config(RepositoryEnv(r"C:\Users\Usuario\Desktop\DB - UPTC\Proyecto_ETL_Dashboard\ETL\src\.env"))

In [28]:
def extraer_db_a_dataframe(tabla):
    host = "localhost"
    password = config("PASSWORD")
    user = "postgres"
    dbname = "eventos"
    port = 5432

    db_uri = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"

    engine = create_engine(db_uri)
    
    sql_query = f"SELECT * FROM {tabla}"

    df = pd.read_sql_query(sql_query, engine)

    return df

# Testeo y Verifico

In [29]:
registro_usuarios = extraer_db_a_dataframe("registro_usuarios")
registro_usuarios.shape

(140, 7)

In [30]:
evento_1 = extraer_db_a_dataframe("evento_1")
evento_1.shape

(117, 8)

In [31]:
evento_2 = extraer_db_a_dataframe("evento_2")
evento_2.shape

(112, 8)

In [32]:
evento_3 = extraer_db_a_dataframe("evento_3")
evento_3.shape

(96, 8)

In [33]:
evento_4 = extraer_db_a_dataframe("evento_4")
evento_4.shape

(80, 8)

In [34]:
evento_5 = extraer_db_a_dataframe("evento_5")
evento_5.shape

(79, 8)

In [35]:
evento_6 = extraer_db_a_dataframe("evento_6")
evento_6.shape

(78, 8)

In [36]:
evento_7 = extraer_db_a_dataframe("evento_7")
evento_7.shape

(65, 8)

In [37]:
evento_8 = extraer_db_a_dataframe("evento_8")
evento_8.shape

(54, 8)

In [38]:
evento_9 = extraer_db_a_dataframe("evento_9")
evento_9.shape

(24, 8)

# Genero archivos de salida

In [39]:
def asistencia_eventos(dataframe_input):
    dataframe_output = pd.merge(registro_usuarios,dataframe_input,how='left',left_on='correo',right_on='Correo electrónico')
    dataframe_output = dataframe_output.dropna(subset=["Correo electrónico"])[["nombre_x","correo","programa_academico","asignatura","estado","Hora a la que se unió","Hora a la que salió","duración"]]
    dataframe_output = dataframe_output[dataframe_output["estado"] != "Falta información"]
    dataframe_output.rename(columns={'nombre_x':'nombre_completo',
                                     'Hora a la que se unió':"hora_entrada",
                                     'Hora a la que salió':"hora_salida"
                                     },inplace=True)
    return dataframe_output[["nombre_completo","correo","programa_academico","asignatura","estado","hora_entrada","hora_salida","duración"]]

In [40]:
asistencia_1 = asistencia_eventos(evento_1)
asistencia_2 = asistencia_eventos(evento_2)
asistencia_3 = asistencia_eventos(evento_3)
asistencia_4 = asistencia_eventos(evento_4)
asistencia_5 = asistencia_eventos(evento_5)
asistencia_6 = asistencia_eventos(evento_6)
asistencia_7 = asistencia_eventos(evento_7)
asistencia_8 = asistencia_eventos(evento_8)
asistencia_9 = asistencia_eventos(evento_9)


In [41]:
def registro_eventos_dataframe(dataframe_uno,dataframe_dos,columna_nueva):
    # Realiza la fusión (merge) con registro_usuarios
    dataframe_output = pd.merge(dataframe_uno, dataframe_dos[["Correo electrónico"]], how='left', left_on='correo', right_on='Correo electrónico')

    # Agrega la columna "evento" con "si" para registros encontrados y "no" para NaN
    dataframe_output[f'{columna_nueva}'] = dataframe_output['Correo electrónico'].apply(lambda x: 'si' if pd.notna(x) else 'no')

    # Filtra las columnas deseadas y elimina filas con estado "Falta información"
    dataframe_output = dataframe_output[dataframe_output["estado"] != "Falta información"]
    dataframe_output = dataframe_output.drop(columns=["Correo electrónico"])

    return dataframe_output

In [42]:
df = registro_eventos_dataframe(registro_usuarios,evento_1,"evento_1")
df = registro_eventos_dataframe(df, evento_2, "evento_2")
df = registro_eventos_dataframe(df, evento_3, "evento_3")
df = registro_eventos_dataframe(df, evento_4, "evento_4")
df = registro_eventos_dataframe(df, evento_5, "evento_5")
df = registro_eventos_dataframe(df, evento_6, "evento_6")
df = registro_eventos_dataframe(df, evento_7, "evento_7")
df = registro_eventos_dataframe(df, evento_8, "evento_8")
df = registro_eventos_dataframe(df, evento_9, "evento_9")
df.drop(columns=["estado","primer_nombre","primer_apellido"],inplace=True)
df.head()

,nombre,correo,programa_academico,asignatura,evento_1,evento_2,evento_3,evento_4,evento_5,evento_6,evento_7,evento_8,evento_9
0,Anderson Daniel Barreto Sierra,anderson.barreto01@uptc.edu.co,Ingeniería de Sistemas y Computación,Electiva III,si,si,si,si,si,si,si,si,si
1,Andrés Felipe Escobar Diaz,andres.escobar01@uptc.edu.co,Ingeniería de Sistemas y Computación,Electiva III,si,si,si,si,si,no,no,no,no
2,Andrés David Reyes Rojas,andres.reyes03@uptc.edu.co,Ingeniería Electrónica,Comunicaciones II,si,no,si,no,no,si,no,no,no
3,Angel Abdías González Molina,angel.gonzalez02@uptc.edu.co,Ingeniería Electrónica,Comunicaciones,si,si,si,si,si,no,si,no,no
4,Angel Leonardo Marciales Supelano,angel.marciales@uptc.edu.co,Ingeniería de Sistemas y Computación,Comunicaciones,si,no,si,si,si,si,si,si,si


In [43]:
asistencia_1.columns

Index(['nombre_completo', 'correo', 'programa_academico', 'asignatura',
       'estado', 'hora_entrada', 'hora_salida', 'duración'],
      dtype='object')

In [44]:
df.programa_academico.unique()

array(['Ingeniería de Sistemas y Computación', 'Ingeniería Electrónica',
       'Gobernación de Boyacá', 'Docente', 'Administrativo'], dtype=object)

In [45]:
profesionales = ["Gobernación de Boyacá","Docente","Administrativo"]
asignaturas_sistemas = df["asignatura"].loc[df["programa_academico"]=="Ingeniería de Sistemas y Computación"].unique()
asignaturas_electronica = df["asignatura"].loc[df["programa_academico"]=="Ingeniería Electrónica"].unique()
asignaturas_profesionales = df["asignatura"].loc[df["programa_academico"].isin(profesionales)].unique()
asignaturas_sistemas,asignaturas_electronica,asignaturas_profesionales

(array(['Electiva III', 'Comunicaciones', 'Transmisión de Datos',
        'Arquitectura de Computadores', 'Matemáticas discretas',
        'Redes de datos', 'Sistemas Operativos'], dtype=object),
 array(['Comunicaciones II', 'Comunicaciones', 'Telemática'], dtype=object),
 array(['Profesionales'], dtype=object))

In [46]:
asistencia_sistemas = (df['programa_academico'] == "Ingeniería de Sistemas y Computación")
sistemas_eventos = df.loc[asistencia_sistemas][["nombre", "correo", "asignatura", "evento_1", "evento_2", "evento_3", "evento_4", "evento_5", "evento_6", "evento_7", "evento_8", "evento_9"]]

asistencia_electronica = (df['programa_academico'] == "Ingeniería Electrónica")
electronica_eventos = df.loc[asistencia_electronica][["nombre", "correo", "asignatura", "evento_1", "evento_2", "evento_3", "evento_4", "evento_5", "evento_6", "evento_7", "evento_8", "evento_9"]]

asistencia_profesionales = (df['programa_academico'].isin(profesionales))
profesionales_eventos = df.loc[asistencia_profesionales][["nombre", "correo", "asignatura", "evento_1", "evento_2", "evento_3", "evento_4", "evento_5", "evento_6", "evento_7", "evento_8", "evento_9"]]


In [47]:
def crea_dataframes(dataframe,asignaturas):
    dataframes = []
    for asignatura in asignaturas:
        df = dataframe.loc[dataframe["asignatura"]==asignatura]
        dataframes.append(df)
    return dataframes

In [48]:
electiva_iii,comunicaciones,transmision,arquitectura,matematicas,redes,sistemas = crea_dataframes(sistemas_eventos, asignaturas_sistemas)
comunicaciones_ii,comunicaciones_electronica,telematica = crea_dataframes(electronica_eventos,asignaturas_electronica)
profesionales = crea_dataframes(profesionales_eventos,asignaturas_profesionales)[0]

## Creación de archivo excel solicitado

In [51]:
with pd.ExcelWriter('../clean_data/proyecto_asistencia.xlsx',engine='openpyxl') as writer:
    sistemas_eventos.to_excel(writer,sheet_name='Ingeniería de Sistemas',index=False)
    electiva_iii.to_excel(writer,sheet_name='Electiva III',index=False)
    comunicaciones.to_excel(writer,sheet_name='Comunicaciones',index=False)
    transmision.to_excel(writer,sheet_name='Transmisión de Datos',index=False)
    arquitectura.to_excel(writer,sheet_name='Arquitectura Computadores',index=False)
    matematicas.to_excel(writer,sheet_name='Matemáticas discretas',index=False)
    redes.to_excel(writer,sheet_name='Redes de datos',index=False)
    sistemas.to_excel(writer,sheet_name='Sistemas Operativos',index=False)
    electronica_eventos.to_excel(writer,sheet_name='Ingeniería Electrónica',index=False)
    comunicaciones_ii.to_excel(writer,sheet_name='Comunicaciones II',index=False)
    comunicaciones_electronica.to_excel(writer,sheet_name='Comunicaciones Elec',index=False)
    telematica.to_excel(writer,sheet_name='Telemática',index=False)
    profesionales.to_excel(writer,sheet_name='Profesionales',index=False)

In [47]:
# Convierte las columnas "hora_entrada" y "hora_salida" en objetos datetime
def tendencia_personas(dataframe):
    dataframe['hora_entrada'] = pd.to_datetime(dataframe['hora_entrada'])
    dataframe['hora_salida'] = pd.to_datetime(dataframe['hora_salida'])

    # Crea una lista de eventos que registra la entrada de una persona como +1 y la salida como -1
    eventos = [(hora, programa, 1) for hora, programa in zip(dataframe['hora_entrada'], dataframe['programa_academico'])] + \
            [(hora, programa, -1) for hora, programa in zip(dataframe['hora_salida'], dataframe['programa_academico'])]

    # Crea un DataFrame a partir de la lista de eventos
    df_eventos = pd.DataFrame(eventos, columns=['hora', 'programa_academico', 'evento'])

    # Ordena los eventos por hora
    df_eventos = df_eventos.sort_values(by='hora')

    # Calcula la cantidad acumulada de personas presentes en cada minuto
    df_eventos['personas_presentes'] = df_eventos.groupby('programa_academico')['evento'].cumsum()

    return df_eventos

In [48]:
evento_06 = tendencia_personas(asistencia_1)
evento_08 = tendencia_personas(asistencia_2)
evento_13 = tendencia_personas(asistencia_3)
evento_15 = tendencia_personas(asistencia_4)

In [49]:
evento_06.pivot_table(index='hora',columns='programa_academico',values='personas_presentes').interpolate(method='linear').fillna(0)

programa_academico,Administrativo,Docente,Gobernación de Boyacá,Ingeniería Electrónica,Ingeniería de Sistemas y Computación
hora,,,,,
2023-09-06 09:45:00,0.0,0.000000,0.000000,0.000000,1.000000
2023-09-06 09:51:00,0.0,1.000000,0.000000,0.000000,1.200000
2023-09-06 09:52:00,0.0,1.088235,0.000000,1.000000,1.400000
2023-09-06 09:53:00,0.0,1.176471,0.000000,2.000000,1.600000
2023-09-06 09:54:00,0.0,1.264706,0.000000,3.000000,1.800000
2023-09-06 09:55:00,0.0,1.352941,0.000000,4.000000,2.000000
2023-09-06 09:56:00,0.0,1.441176,0.000000,5.000000,3.000000
2023-09-06 09:57:00,0.0,1.529412,0.000000,6.000000,4.000000
2023-09-06 09:58:00,0.0,1.617647,0.000000,7.000000,7.000000


In [50]:
df.loc[df["programa_academico"]=="Docente"]

,nombre,correo,programa_academico,asignatura,evento_1,evento_2,evento_3,evento_4,evento_5,evento_6,evento_7,evento_8,evento_9
42,Frey Alfonso Santamaría Buitrago,frey.santamaria@uptc.edu.co,Docente,Profesionales,si,si,si,si,si,si,si,si,si
49,Ingrid Ortiz,ingrid.ortiz05@uptc.edu.co,Docente,Profesionales,no,no,si,no,no,no,si,si,si
50,Jairo Alonso Mesa Lara,jairo.mesa@uptc.edu.co,Docente,Profesionales,no,no,no,no,no,no,no,no,no
51,Jaraquesoler,jaraquesoler@gmail.com,Docente,Profesionales,no,no,no,no,no,no,no,no,no
56,Jenny Amparo Rosales Agredo,jenny.rosales@uptc.edu.co,Docente,Profesionales,si,si,si,si,si,si,si,si,si
96,Luz Santamaría Granados,luz.santamaria@uptc.edu.co,Docente,Profesionales,si,si,no,no,si,no,no,no,no
97,Luz Santamaría Granados,luz.santamaria@uptc.edu.co,Docente,Profesionales,si,si,no,no,si,no,no,no,no
108,Miguel Angel Mendoza Moreno,miguel.mendoza@uptc.edu.co,Docente,Profesionales,no,no,no,no,no,no,no,no,no
115,Oscar Fernando Vera Cely,oscar.vera@uptc.edu.co,Docente,Profesionales,si,si,si,si,si,si,si,no,no


In [51]:
people_tendency = px.line(data_frame=evento_06,x='hora',y='personas_presentes',
                          title="Tendencia de personas por programa",template='plotly_dark',
                          color='programa_academico')
people_tendency.update_layout(title=dict(font=dict(family='JetBrains Mono',size=30)),
                            xaxis=dict(nticks=23))
people_tendency.show()

In [114]:
personas_por_asignatura = asistencia_dia_06.groupby(["programa_academico","asignatura"]).size().reset_index(name="cantidad_personas")

fig = px.pie(personas_por_asignatura,
             names='asignatura',
             values='cantidad_personas',
             title='Cantidad de personas por asignatura',
             color='programa_academico',
             hole=0.3,
             template='plotly_dark')

fig.update_traces(textinfo='percent+label',
                  textposition='outside',  # Colocar el texto dentro del pie
                  insidetextorientation='radial',  # Orientación radial para el texto interior
                  textfont=dict(color='white', size=14))

# Configurar la posición del título
fig.update_layout(
    title='Cantidad de Personas por Asignatura',
    title_x=0.5,  # Centrar horizontalmente el título (0.5 = centro)
    title_y=0.95,  # Ajustar la posición vertical del título (0.95 = cerca de la parte superior)
)

# Configurar el tamaño del gráfico
fig.update_layout(
    width=1000,  # Cambiar el ancho del gráfico
    height=600  # Cambiar la altura del gráfico
)


fig.show()

In [115]:
# Duración promedio
duracion_promedio = round(asistencia_dia_06["duración"].mean(),2)
duracion_promedio

108.61

In [129]:
# No asistieron al evento del día
dia_06 = df["evento_06"]=="si"
df[["nombre","correo","programa_academico","asignatura"]].loc[~dia_06]

,nombre,correo,programa_academico,asignatura
20,Daniel Andres Cardenas Arevalo,daniel.cardenas01@uptc.edu.co,Ingeniería de Sistemas y Computación,Electiva III
40,Fredy Andrés Murcia Currea,fandresmc97@gmail.com,Gobernación de Boyacá,Profesionales
48,Ingrid Ortiz,ingrid.ortiz05@uptc.edu.co,Docente,Profesionales
49,Jairo Alonso Mesa Lara,jairo.mesa@uptc.edu.co,Docente,Profesionales
50,Jaraquesoler,jaraquesoler@gmail.com,Docente,Profesionales
55,Jeisson Alexander Prieto Torres,jeisson.prieto@uptc.edu.co,Ingeniería de Sistemas y Computación,Comunicaciones
66,Jorge Steven Ayala Benitez,jorge.ayala01@uptc.edu.co,Ingeniería de Sistemas y Computación,Arquitectura de Computadores
73,Juan Esteban González Bermúdez,juan.gonzalez37@uptc.edu.co,Ingeniería de Sistemas y Computación,Comunicaciones
74,Juan Sebastian Martinez Noreña,juan.martinez30@uptc.edu.co,Ingeniería de Sistemas y Computación,Comunicaciones
78,Julián David Cely Forero,julian.cely@uptc.edu.co,Ingeniería Electrónica,Telemática
